In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics

print('lebo')

lebo


In [2]:
df = pd.read_csv('agaricus-lepiota.data')
df.head()

,p,x,s,n,t,p.1,f,c,n.1,k,...,s.2,w,w.1,p.2,w.2,o,p.3,k.1,s.3,u
0,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
1,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
2,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
3,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
4,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g


In [3]:
df.columns

Index(['p', 'x', 's', 'n', 't', 'p.1', 'f', 'c', 'n.1', 'k', 'e', 'e.1', 's.1',
       's.2', 'w', 'w.1', 'p.2', 'w.2', 'o', 'p.3', 'k.1', 's.3', 'u'],
      dtype='object')

#### renaming columns

In [4]:
df = df.rename(columns= {'p':'classes', 'x':'cap-shape', 's':'cap-surface', 'n':'cap-color', 't':'bruises?', 
                         'p.1':'odor', 'f':'gill-attachment', 
                         'c':'gill-spacing', 'n.1':'gill-size', 'k':'gill-color', 'e':'stalk-shape', 'e.1':'stalk-root', 
                         's.1':'stalk-surface-above-ring','s.2':'stalk-surface-below-ring', 'w':'stalk-color-above-ring', 
                         'w.1':'stalk-color-below-ring', 'p.2':'veil-type', 
                         'w.2':'veil-color', 'o':'ring-number', 'p.3':'ring-type', 'k.1':'spore-print-color',
                         's.3':'population', 'u':'habitat'})

In [5]:
df.head()

,classes,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
1,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
2,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
3,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
4,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g


In [6]:
df = df.replace({'?':np.NaN})

In [7]:
df.isnull().sum()

classes                        0
cap-shape                      0
cap-surface                    0
cap-color                      0
bruises?                       0
odor                           0
gill-attachment                0
gill-spacing                   0
gill-size                      0
gill-color                     0
stalk-shape                    0
stalk-root                  2480
stalk-surface-above-ring       0
stalk-surface-below-ring       0
stalk-color-above-ring         0
stalk-color-below-ring         0
veil-type                      0
veil-color                     0
ring-number                    0
ring-type                      0
spore-print-color              0
population                     0
habitat                        0
dtype: int64

In [8]:
print(df['stalk-root'].value_counts())

b    3776
e    1119
c     556
r     192
Name: stalk-root, dtype: int64


*There is various ways to handle missing values of categorical ways.*

    -Ignore observations of missing values if we are dealing with large data sets and less number of records has missing values
    -Ignore variable, if it is not significant
    -Develop model to predict missing values
    -Treat missing data as just another category

In [9]:
df['stalk-root'] = df['stalk-root'].replace(np.NaN,'n')

In [10]:
print(df['stalk-root'].value_counts())

b    3776
n    2480
e    1119
c     556
r     192
Name: stalk-root, dtype: int64


In [11]:
df['classes'].value_counts()

e    4208
p    3915
Name: classes, dtype: int64

In [12]:
data = df.drop('classes',axis=1)

In [13]:
data = pd.get_dummies(data,drop_first = True)


### Use a decision tree model to predict whether mushrooms are poisonous or edible

### 1. Split your data into train and test sets

In [14]:
X = data
y =df.classes

In [15]:
X

,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_g,cap-surface_s,cap-surface_y,cap-color_c,cap-color_e,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,1,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
3,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8118,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8119,0,0,0,0,1,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
8120,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8121,0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [17]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(6498, 95)
(6498,)
(1625, 95)
(1625,)


### 2. Get basic descriptive statistics for the training data and check for missing and incorrect or extreme values.

In [18]:
X_train.describe()

,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_g,cap-surface_s,cap-surface_y,cap-color_c,cap-color_e,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
count,6498.000000,6498.000000,6498.000000,6498.000000,6498.000000,6498.000000,6498.000000,6498.000000,6498.000000,6498.000000,...,6498.000000,6498.000000,6498.000000,6498.000000,6498.000000,6498.000000,6498.000000,6498.000000,6498.000000,6498.000000
mean,0.000616,0.392582,0.103109,0.003693,0.447215,0.000462,0.315943,0.398430,0.005848,0.184980,...,0.050939,0.150816,0.494460,0.212219,0.267159,0.102493,0.035549,0.141890,0.044321,0.024161
std,0.024805,0.488363,0.304124,0.060666,0.497244,0.021483,0.464926,0.489613,0.076254,0.388311,...,0.219890,0.357897,0.500008,0.408911,0.442510,0.303319,0.185178,0.348964,0.205824,0.153562
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
feature_list = list(X_train.columns)

In [20]:
model = DecisionTreeClassifier(random_state=47)
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=47, splitter='best')

### 3. What are the factors that predict whether a mushroom is poisonous? 

In [21]:
importances = list(model.feature_importances_)

In [22]:
feature_importances = list(zip(feature_list, importances))
features_importance = pd.DataFrame(feature_importances,columns = ['features','importance'])
len(features_importance)


95

Features that are not contributing any importance at all.

In [23]:
n_features = features_importance[features_importance['importance']==0.000]
not_important = features_importance[features_importance['importance']==0.000].index
len(n_features)


83

Features that do contribute to the model

In [24]:
features_importance = features_importance.drop(not_important,axis=0)
print(features_importance)

                      features  importance
0                  cap-shape_c    0.000615
5                cap-surface_g    0.001841
17                  bruises?_t    0.011543
20                      odor_l    0.009680
22                      odor_n    0.620739
27              gill-spacing_w    0.002514
28                 gill-size_n    0.005977
41                stalk-root_c    0.178122
50  stalk-surface-below-ring_y    0.094289
74                 ring-type_n    0.015606
80         spore-print-color_r    0.034725
81         spore-print-color_u    0.024349


predictions

In [25]:
y_predict = model.predict(X_test)
y_predict

array(['p', 'e', 'e', ..., 'e', 'p', 'p'], dtype=object)

In [26]:
X_train1 = X_train[['cap-surface_g','odor_l','odor_n','gill-spacing_w','gill-size_n','stalk-shape_t','stalk-root_c',
              'stalk-surface-below-ring_y','stalk-color-below-ring_y','ring-number_o','spore-print-color_r',
              'spore-print-color_u','population_c']]

In [27]:
X_test1 = X_test[['cap-surface_g','odor_l','odor_n','gill-spacing_w','gill-size_n','stalk-shape_t','stalk-root_c',
              'stalk-surface-below-ring_y','stalk-color-below-ring_y','ring-number_o','spore-print-color_r',
              'spore-print-color_u','population_c']]

In [28]:
X_train1.shape

(6498, 13)

In [29]:
y_train.shape

(6498,)

In [30]:
model1 = DecisionTreeClassifier(random_state=47)
model1.fit(X_train1, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=47, splitter='best')

In [31]:
y_predict1 = model1.predict(X_test1)

# Evaluating Model

### 4. Report the accuracy of your model on the training set and on the test set. 

In [32]:
print("Accuracy:",metrics.accuracy_score(y_test, y_predict))
print("Accuracy:",metrics.accuracy_score(y_test, y_predict1))

Accuracy: 1.0
Accuracy: 1.0


In [33]:
confusion_matrix(y_test, y_predict)

array([[850,   0],
       [  0, 775]])

In [34]:
confusion_matrix(y_test, y_predict1)

array([[850,   0],
       [  0, 775]])

In [35]:
print("Accuracy of Decision Tree classifier: ", model.score(X_test,y_predict))

Accuracy of Decision Tree classifier:  1.0


In [36]:
print("Accuracy of Decision Tree classifier: ", model.score(X_test,y_predict1))

Accuracy of Decision Tree classifier:  1.0


In [37]:
y_predict = pd.DataFrame(y_predict,columns=['y_predict'])
outputs= y_test.merge([y_predict])
print(len(y_test))
print(len(y_predict))


AttributeError: 'Series' object has no attribute 'merge'

In [ ]:
y_predict


In [ ]:
y_predict

In [ ]:
outputs